In [2]:
# 约瑟夫环
# 创建两个类，面向人和环
# 指定起始位置，间隔step的人出列
# 容器具有增、删、查、改的功能

import os
import os.path
import csv
import zipfile
import copy

class Person(object):
    def __init__(self, name, age):
        self.name = name
        if age < 0:
            raise ValueError('The value of age can be less than 0') 
        self.age = age

class Reader:
    def __iter__(self):
        raise NotImplementedError

    def __next__(self):
        raise NotImplementedError

class TxtReader(Reader):
    def __init__(self,path):
        self.file = open(path)

    def __del__(self):
        if self.file:
            self.file.close()

    def __iter__(self):
        return self
    
    def __next__(self):
        row = self.file.readline()
        if not row:
            self.file.close()
            raise StopIteration
        line = row.strip().split(',')
        name = line[0]
        try:
            age = int(line[1])
        except ValueError:
            age = 0
        return Person(name, age)

class CSVReader(Reader):
    def __init__(self, path):
        self.csv_file = open(path)
        csv_reader = csv.reader(self.csv_file)
        self.it = csv_reader.__iter__()

    def __del__(self):
        if self.csv_file:
            self.csv_file.close()

    def __iter__(self):
        return self

    def __next__(self):
        try:
            line = self.it.__next__()
        except StopIteration:
            self.csv_file.close()
            raise StopIteration
        name = line[0]
        try:
            age = int(line[1])
        except ValueError:
            age = 0
        return Person(name, age)

class TxtZipReader(TxtReader):
    def __init__(self, path, target_file):
        with zipfile.ZipFile(path) as zip_file:
            file_path = zip_file.extract(target_file)
        TxtReader.__init__(self, file_path)

class CSVZipReader(CSVReader):
    def __init__(self, path, target_file):
        with zipfile.ZipFile(path) as zip_file:
            file_path = zip_file.extract(target_file)
        CSVReader.__init__(self, file_path)

class Ring:
    max_size = 100

    def __init__(self, reader = None):
        self.start = 0
        self.step = 1
        self.people = []
        if reader:
            for each in reader:
                self.people.append(each)
    
     # 增功能
    def append(self, obj):
        if(len(self.people) > Ring.max_size):
            raise Exception("exceed the max cap of people.")
        self.people.append(obj)
   
    # 删功能
    def pop(self):
        self.people.pop(obj)
    
    # 查找
    def query_list(self):
        return self.people
    
    def reset(self):  
        self.current_id = self.start - 1
        self.temp = copy.copy(self.people)
        self.length = len(self.people)

    # 生成器迭代输出
    def iter(self):
        if len(self.temp) == 0:
            return None

        for i in range(len(self.temp)):
            self.current_id = (self.current_id + self.step - 1) % len(self.temp)
            index = self.temp.pop(self.current_id)
            yield index

    # @classmethod
    # def create_from_txt(cls, path):
    #     obj = cls()

    #     with open(path, 'r') as txt_file:
    #         for line in txt_file:
    #             temp = line.strip().split(',')
    #             obj.append(Person(temp[0], temp[1]))
        
    #     return obj
    
    # @classmethod
    # def create_from_csv(cls, path): 
    #     obj = cls()
         
    #     with open(path,'r') as csv_file:
    #         csv_reader = csv.reader(csv_file)
    #         for line in csv_reader:
    #             obj.append(Person(line[0], line[1]))
        
    #     return obj

    # @classmethod
    # def create_from_zip(cls, path, target_file):
    #     with zipfile.ZipFile(path, 'r') as f:
    #         open_file = f.extract(target_file)
            
    #         if '.txt' in target_file:
    #             obj = Ring.create_from_txt(open_file)
            
    #         if '.csv' in target_file:
    #             obj = Ring.create_from_csv(open_file)

    #         return obj


In [17]:
txt_reader = TxtReader('people.txt')
ring = Ring(txt_reader)
for each in ring.people:
    print("name:{}, age:{}" .format(each.name, each.age))

print('-'*30)

ring.start = 2
ring.step = 3
ring.reset()
generator = ring.iter()
for i in range(ring.length):
    index = next(generator)
    if index:
        print("第{}个出列的人：{}；年龄：{}". format(i+1, index.name, index.age))

name:"John", age:18
name:"Bob", age:19
name:"Jerry", age:20
name:"Mark", age:21
name:"Jack", age:22
name:"Neil", age:23
name:"Oscar", age:24
------------------------------
第1个出列的人："Mark"；年龄：21
第2个出列的人："Oscar"；年龄：24
第3个出列的人："Jerry"；年龄：20
第4个出列的人："John"；年龄：18
第5个出列的人："Neil"；年龄：23
第6个出列的人："Bob"；年龄：19
第7个出列的人："Jack"；年龄：22


In [15]:
csv_reader = CSVReader('people.csv')
ring = Ring(csv_reader)

ring.start = 2
ring.step = 3
ring.reset()
generator = ring.iter()
for i in range(ring.length):
    index = next(generator)
    if index:
        print("第{}个出列的人：{}；年龄：{}". format(i+1, index.name, index.age))

第1个出列的人：Mark；年龄：21
第2个出列的人：Oscar；年龄：24
第3个出列的人：Jerry；年龄：20
第4个出列的人：John；年龄：18
第5个出列的人：Neil；年龄：23
第6个出列的人：Bob；年龄：19
第7个出列的人：Jack；年龄：22


In [9]:
txt_zipreader = TxtZipReader('people.zip', 'people.txt')
ring = Ring(txt_zipreader)
for each in ring.people:
    print("name:{}, age:{}" .format(each.name, each.age))

name:"John", age:18
name:"Bob", age:19
name:"Jerry", age:20
name:"Mark", age:21
name:"Jack", age:22
name:"Neil", age:23
name:"Oscar", age:24


In [8]:
csv_zipreader = CSVZipReader('people.zip', 'people.csv')
ring = Ring(csv_zipreader)
for each in ring.people:
    print("name:{}, age:{}" .format(each.name, each.age))

name:John, age:18
name:Bob, age:19
name:Jerry, age:20
name:Mark, age:21
name:Jack, age:22
name:Neil, age:23
name:Oscar, age:24
